In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Intellectual_Disability"
cohort = "GSE273850"

# Input paths
in_trait_dir = "../../input/GEO/Intellectual_Disability"
in_cohort_dir = "../../input/GEO/Intellectual_Disability/GSE273850"

# Output paths
out_data_file = "../../output/preprocess/Intellectual_Disability/GSE273850.csv"
out_gene_data_file = "../../output/preprocess/Intellectual_Disability/gene_data/GSE273850.csv"
out_clinical_data_file = "../../output/preprocess/Intellectual_Disability/clinical_data/GSE273850.csv"
json_path = "../../output/preprocess/Intellectual_Disability/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from human trisomy 21 and euploid fibroblasts, iPSCs, and neural progenitor cells"
!Series_summary	"Trisomy of human chromosome 21 (T21) gives rise to Down syndrome, the most frequent life-born autosomal aneuploidy. To enable in vitro analyses of the cellular and moelcular mechanisms leading to the neurological alterations associated with T21, we created and characterized a panel of genomically diverse T21 and euploid induced pluripotent stem cells (iPSCs) from fibroblasts obtained from the Coriell Institute for Biomedical Research, and we then differentiated these iPSCs into neural progenitor cells (NPCs)."
!Series_summary	"Microarray transcriptomic analyses were performed on this panel of fibroblasts, iPSCs, and NPCs, identifying genes and pathways that were altered in T21 lines relative to euploid as well as genes and pathways in NPCs that showed inter-individual variability."
!Series_overall_design	"This study used cell lines o

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# STEP 1: Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data 
# from microarray transcriptomic analyses performed on fibroblasts, iPSCs, and NPCs
is_gene_available = True

# STEP 2: Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Intellectual Disability), from the data we see trisomy 21 (Down syndrome) which is linked to intellectual disability
trait_row = 0  # 'genotype: T21' vs 'genotype: Euploid'

# Age is not available in the sample characteristics
age_row = None

# Gender information is available 
gender_row = 1  # 'Sex: male' vs 'Sex: female'

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trisomy 21 status to binary value for intellectual disability"""
    if value is None:
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() == "t21":
        return 1  # T21 (Down syndrome) is associated with intellectual disability
    elif value.lower() == "euploid":
        return 0  # Euploid (normal) is the control
    else:
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() == "male":
        return 1
    elif value.lower() == "female":
        return 0
    else:
        return None

# STEP 3: Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# STEP 4: Clinical Feature Extraction
if trait_row is not None:
    # Use the library function to extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    print("Preview of clinical data:")
    print(preview_df(clinical_df))
    
    # Save clinical data to CSV
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM8438101': [1.0, 1.0], 'GSM8438102': [1.0, 0.0], 'GSM8438103': [1.0, 1.0], 'GSM8438104': [1.0, 0.0], 'GSM8438105': [1.0, 1.0], 'GSM8438106': [1.0, 0.0], 'GSM8438107': [1.0, 1.0], 'GSM8438108': [0.0, 0.0], 'GSM8438109': [0.0, 1.0], 'GSM8438110': [0.0, 0.0], 'GSM8438111': [1.0, 1.0], 'GSM8438112': [0.0, 1.0], 'GSM8438113': [0.0, 1.0], 'GSM8438114': [0.0, 1.0], 'GSM8438115': [0.0, 1.0], 'GSM8438116': [0.0, 0.0], 'GSM8438117': [0.0, 1.0], 'GSM8438118': [1.0, 1.0], 'GSM8438119': [1.0, 0.0], 'GSM8438120': [0.0, 0.0], 'GSM8438121': [1.0, 1.0], 'GSM8438122': [1.0, 0.0], 'GSM8438123': [1.0, 1.0], 'GSM8438124': [1.0, 0.0], 'GSM8438125': [1.0, 1.0], 'GSM8438126': [0.0, 0.0], 'GSM8438127': [0.0, 1.0], 'GSM8438128': [0.0, 0.0], 'GSM8438129': [1.0, 1.0], 'GSM8438130': [0.0, 1.0], 'GSM8438131': [0.0, 1.0], 'GSM8438132': [0.0, 1.0], 'GSM8438133': [0.0, 1.0], 'GSM8438134': [0.0, 0.0], 'GSM8438135': [0.0, 1.0], 'GSM8438136': [1.0, 1.0], 'GSM8438137': [1.0, 0.0], 'GSM8438138

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 21448 rows
First 20 gene IDs:
Index(['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1',
       'TC0100006480.hg.1', 'TC0100006483.hg.1', 'TC0100006486.hg.1',
       'TC0100006490.hg.1', 'TC0100006492.hg.1', 'TC0100006494.hg.1',
       'TC0100006497.hg.1', 'TC0100006499.hg.1', 'TC0100006501.hg.1',
       'TC0100006502.hg.1', 'TC0100006514.hg.1', 'TC0100006516.hg.1',
       'TC0100006517.hg.1', 'TC0100006524.hg.1', 'TC0100006540.hg.1',
       'TC0100006548.hg.1', 'TC0100006550.hg.1'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# The identifiers "TC0100006437.hg.1", "TC0100006476.hg.1", etc. appear to be probe IDs
# from a microarray platform rather than standard human gene symbols.
# These are likely Affymetrix transcript cluster IDs that need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1121088 rows

Gene annotation preview (first few rows):
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 // ENSEMBL // olfactory receptor, family 4, subfamily F, member

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Examine gene identifiers and annotations
# Based on the previous outputs, we can see the gene expression data identifiers (e.g., TC0100006437.hg.1)
# match with the 'ID' column in the gene annotation dataframe.
# The SPOT_ID.1 column contains gene name information embedded with RefSeq, ENSEMBL, etc. annotations

# First, ensure we have the gene expression data retrieved from the matrix file
gene_data = get_genetic_data(matrix_file)
print(f"Retrieved gene expression data with {gene_data.shape[0]} rows and {gene_data.shape[1]} columns")

# 2. Create gene mapping dataframe
print("Creating gene mapping dataframe...")

# Use the library function to extract gene symbols from SPOT_ID.1 field
gene_annotation['Gene'] = gene_annotation['SPOT_ID.1'].apply(extract_human_gene_symbols)

# Filter out common database/platform terms that are not actual gene symbols
def filter_gene_symbols(symbols):
    if not symbols:
        return []
    filtered = []
    common_terms = {'ENSEMBL', 'UCSC', 'CCDS', 'HGNC', 'NONCODE', 'MGC', 'IMAGE', 
                   'ANNOTATED', 'CDS', 'INTERNAL', 'OVCODE', 'OVERLAPTX', 'OVEXON', 'UTR3', 'ID'}
    for symbol in symbols:
        # Keep only if it's not in our list of common terms and looks like a gene symbol
        if symbol not in common_terms and re.match(r'^[A-Z0-9-]{2,15}$', symbol):
            filtered.append(symbol)
    return filtered

# Apply filtering to remove non-gene terms
gene_annotation['Gene'] = gene_annotation['Gene'].apply(filter_gene_symbols)

# Create mapping dataframe with ID and Gene columns
mapping_df = gene_annotation[['ID', 'Gene']].copy()
mapping_df = mapping_df.dropna(subset=['Gene'])

# Filter to include only rows where Gene is not an empty list
mapping_df = mapping_df[mapping_df['Gene'].apply(lambda x: len(x) > 0)]

print(f"Created mapping dataframe with {len(mapping_df)} rows")
print("Sample of mapping data:")
print(preview_df(mapping_df))

# 3. Convert probe-level measurements to gene expression data
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Check the result
if gene_data.empty:
    print("WARNING: No gene expression data was generated after mapping. This indicates a problem with the gene mapping process.")
    is_gene_available = False
else:
    print(f"Successfully generated gene expression data with {len(gene_data.index)} genes")
    print(f"First 10 gene symbols: {list(gene_data.index[:10])}")
    is_gene_available = True

# Normalize gene symbols to standard NCBI Gene symbols
if is_gene_available:
    print("\nNormalizing gene symbols using NCBI Gene database information...")
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(gene_data.index)} genes")
    print(f"First 10 normalized gene symbols: {list(gene_data.index[:10])}")
    
    # Save processed gene data
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")


Retrieved gene expression data with 21448 rows and 51 columns
Creating gene mapping dataframe...


Created mapping dataframe with 23477 rows
Sample of mapping data:
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'Gene': [['OR4F5', 'CCDS30547'], ['SAMD11', 'BC024295', 'BC033213', 'CCDS2'], ['KLHL17', 'BC166618', 'CCDS30550'], ['PLEKHN1', 'BC101386', 'BC101387', 'CCDS4', 'CCDS53256'], ['ISG15', 'BC009507', 'CCDS6']]}

Converting probe-level measurements to gene expression data...


### Step 7: Data Normalization and Linking

In [8]:
# 1-2. Check if we need to fix the gene mapping issue from the previous step
print("Attempting to fix gene mapping issues...")
try:
    # Re-extract the gene expression data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    # Create a simpler mapping for genes based on the SPOT_ID.1 field
    # This approach extracts potential gene symbols more aggressively
    def extract_likely_gene(text):
        if not isinstance(text, str):
            return []
        # Use a pattern that's more likely to capture gene names from the RefSeq annotations
        matches = re.findall(r'([A-Z0-9]+)(?:\s+//\s+RefSeq|\[Source:HGNC Symbol;Acc:HGNC)', text)
        return list(dict.fromkeys(matches))  # Remove duplicates while preserving order
    
    # Apply the new extraction method
    gene_annotation['Gene'] = gene_annotation['SPOT_ID.1'].apply(extract_likely_gene)
    
    # Create mapping dataframe with ID and Gene columns
    mapping_df = gene_annotation[['ID', 'Gene']].copy()
    mapping_df = mapping_df[mapping_df['Gene'].apply(lambda x: len(x) > 0)]
    
    print(f"Created new mapping dataframe with {len(mapping_df)} rows")
    print("Sample of new mapping data:")
    print(preview_df(mapping_df.head(3)))
    
    # Apply the mapping with the new approach
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    
    # Check if we now have gene expression data
    if gene_data.empty:
        print("Still no gene expression data after improved mapping. Trying alternative method...")
        
        # Last resort: extract genes from column annotations directly
        print("Creating a direct gene mapping from annotation...")
        simple_mapping = []
        for idx, row in gene_annotation.iterrows():
            spot_id = row['SPOT_ID.1']
            if isinstance(spot_id, str):
                # Look for Homo sapiens gene name patterns
                genes = re.findall(r'Homo sapiens ([A-Z0-9-]{2,15})', spot_id)
                if genes:
                    simple_mapping.append({'ID': row['ID'], 'Gene': genes})
        
        if simple_mapping:
            mapping_df = pd.DataFrame(simple_mapping)
            gene_data = apply_gene_mapping(gene_data, mapping_df)
            print(f"Direct mapping created with {len(mapping_df)} probe-gene pairs")
    
    # Verify if we now have gene data
    if gene_data.empty:
        print("WARNING: Gene mapping still unsuccessful. Cannot proceed with gene expression analysis.")
        is_gene_available = False
    else:
        print(f"Mapping successful! Generated gene expression data with {len(gene_data.index)} genes")
        print(f"First 10 gene symbols: {list(gene_data.index[:10])}")
        is_gene_available = True
        
        # Normalize gene symbols to standard NCBI Gene symbols
        print("\nNormalizing gene symbols using NCBI Gene database information...")
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"After normalization: {len(normalized_gene_data.index)} genes")
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        
        # Save normalized gene data
        normalized_gene_data.to_csv(out_gene_data_file)
        print(f"Normalized gene data saved to {out_gene_data_file}")
        print(f"Normalized gene data shape: {normalized_gene_data.shape}")
except Exception as e:
    print(f"Error fixing gene mapping: {e}")
    is_gene_available = False
    normalized_gene_data = pd.DataFrame()

# 3. Load clinical data from file and link with genetic data
print("\nLoading clinical data from file...")
try:
    # Load the previously saved clinical data
    clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
    
    # Determine available clinical features
    clinical_features = clinical_df.index.tolist()
    print(f"Available clinical features: {clinical_features}")
    
    # Set is_trait_available based on whether the clinical data contains the trait
    is_trait_available = trait in clinical_features
    
    # Transpose clinical data for linking (samples as rows)
    clinical_df_t = clinical_df.T
    
    # Only attempt linking if we have gene data
    if is_gene_available and not normalized_gene_data.empty:
        print("\nLinking clinical and genetic data...")
        
        # Link clinical and genetic data - transpose gene data so samples are rows
        linked_data = pd.merge(clinical_df_t, normalized_gene_data.T, 
                              left_index=True, right_index=True, how='inner')
        print(f"Linked data shape: {linked_data.shape}")
        
        # 4. Handle missing values systematically
        print("\nHandling missing values...")
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values, data shape: {linked_data.shape}")
        
        # 5. Determine whether features are biased
        print("\nChecking for bias in features...")
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    else:
        print("Gene expression data not available. Cannot link with clinical data.")
        linked_data = clinical_df_t.copy()
        is_biased = True  # Set as biased since we don't have gene data
    
    # 6. Final validation and save metadata
    print("\nPerforming final validation...")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=is_biased,
        df=linked_data,
        note="Trisomy 21 (Down syndrome) patients as intellectual disability cases"
    )
    
    # 7. Save the linked data if usable
    if is_usable:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        
        # Save linked data
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print(f"Dataset not usable for {trait} association studies. Data not saved.")

except Exception as e:
    print(f"Error in data linking or processing: {e}")
    # Create a minimal dataframe for validation purposes
    linked_data = pd.DataFrame({trait: [0, 1]})
    
    # Perform final validation with appropriate flags
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available, 
        is_trait_available=is_trait_available,
        is_biased=True,  # Not relevant since data isn't usable
        df=linked_data,
        note="Failed to link gene and clinical data: " + str(e)
    )
    print(f"Dataset usability: {is_usable}")

Attempting to fix gene mapping issues...


Created new mapping dataframe with 19524 rows
Sample of new mapping data:
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1'], 'Gene': [['001005484'], ['152486'], ['198317']]}
Still no gene expression data after improved mapping. Trying alternative method...
Creating a direct gene mapping from annotation...


Direct mapping created with 4261 probe-gene pairs

Loading clinical data from file...
Available clinical features: ['Intellectual_Disability', 'Gender']
Gene expression data not available. Cannot link with clinical data.

Performing final validation...
Abnormality detected in the cohort: GSE273850. Preprocessing failed.
Dataset not usable for Intellectual_Disability association studies. Data not saved.
